In [1]:
# Импортируем библиотеку, соответствующую типу нашей базы данных 
import sqlite3
import xlrd, xlwt
from tqdm import tqdm_notebook
import csv
from datetime import datetime
import numpy
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.utils import np_utils
import numpy as np
import pandas


# Создаем соединение с базой данных
conn = sqlite3.connect('C:/Users/Users/Maxim/Nero/Chinook_Sqlite.sqlite')

# Создаем курсор
cursor2 = conn.cursor()
cursor3 = conn.cursor()
cursor4 = conn.cursor()
fmt = '%Y-%m-%d %H:%M:%S'
fmt2 = '%d.%m.%Y %H:%M:%S'

# Делаем SELECT запрос
cursor2.execute("""SELECT * FROM test ORDER BY difference""")
results2 = cursor2.fetchall()

ZZZ=0

Using TensorFlow backend.


In [2]:
def csv_writer(data, path):
    outfile = open(path,'w')
    out = csv.writer(outfile)
    out.writerows(map(lambda x: [x], data))
    outfile.close()

In [ ]:
# Счётчики
i = 0
o = 0
b = 0

Z = np.zeros((3001,len(results2)))
for i in tqdm_notebook(range(len(results2))):
    cursor3.execute("SELECT * FROM ner2 WHERE data= :data", {"data":results2[i][2]})
    results3 = cursor3.fetchall()
    if (not(results3 == [])):
        cursor4.execute("""SELECT * FROM ner2 WHERE id_tweet< :idwt ORDER BY id_tweet DESC LIMIT 1500""", {"idwt":results3[0][1]})
        results4 = cursor4.fetchall()
        a = 0
        if len(results4) == 1500:
            while a < 1500:
                Z[2 * a][o] = results4[a][3]
                Z[2 * a + 1][o] = results4[a][4]
                a += 1
                b += 1
            Z[3000][o] = results3[0][5]
        o += 1
    i += 1
#path = 'C:/Users/Maxim/Nero/Ner/Ner'+str(o)+'.csv' Печать в файл
#csv_writer(Z,path)
print(i)
print(o)

In [ ]:
Z=Z.T
X_train = Z[:,0:2999]
X_test = Z[:,3000]
one_hot_labels = keras.utils.to_categorical(X_test, num_classes=3)

In [ ]:
model = Sequential()
model.add(Dense(1500, activation='relu', input_dim=2999))
model.add(Dropout(0.5))
model.add(Dense(750, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, one_hot_labels, epochs=1, batch_size=3)

In [ ]:
from keras.models import load_model
model.save('C:/Users/Users/Maxim/Nero/my_model.h5')  # creates a HDF5 file 'my_model.h5'
del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
model = load_model('C:/Users/Users/Maxim/Nero/my_model.h5')